---
# **LAB 3 - CUDA Execution Model**
---

# ▶️ CUDA tools...

In [1]:
!nvidia-smi

Thu Jan 22 15:05:32 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
import numba
from numba import cuda
import warnings
warnings.filterwarnings("ignore")

print(np.__version__)
print(numba.__version__)

cuda.detect()



2.0.2
0.60.0
Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 4
                              PCI Bus ID: 0
                                    UUID: GPU-324390f3-6c41-94bc-9b89-6f45e2b8e3c3
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [3]:
# Suppress Numba deprecation and performance warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

Utils for compiling and running Numba CUDA code.

In [4]:
from numba import cuda

def mem_snapshot():
    free, total = cuda.current_context().get_memory_info()
    return total - free, free, total

# Quick device spec report (Numba)
def print_device_info():
    dev = cuda.get_current_device()   # raises if no CUDA device
    ctx = cuda.current_context()

    print("Device object repr:", dev)
    print("Device name:          ", getattr(dev, "name", "<unknown>"))
    print("Compute capability:   ", getattr(dev, "compute_capability", "<unknown>"))

    # Common numeric properties (use getattr to avoid attribute errors)
    props = {
        "  multi_processor_count": ["MULTIPROCESSOR_COUNT"],
        "  max_threads_per_block": ["MAX_THREADS_PER_BLOCK"],
        "  max_block_dim_x":       ["MAX_BLOCK_DIM_X"],
        "  total_memory (bytes)":  ["total_memory"],
        "  shared_memory_per_block (bytes)": ["MAX_SHARED_MEMORY_PER_BLOCK"],
        "  warp_size":             ["WARP_SIZE"],
    }

    for label, keys in props.items():
        val = None
        for k in keys:
            val = getattr(dev, k, None)
            if val is not None:
                break
        print(f"{label:33}: {val}")

print_device_info()

Device object repr: <CUDA device 0 'b'Tesla T4''>
Device name:           b'Tesla T4'
Compute capability:    (7, 5)
  multi_processor_count          : 40
  max_threads_per_block          : 1024
  max_block_dim_x                : 1024
  total_memory (bytes)           : None
  shared_memory_per_block (bytes): 49152
  warp_size                      : 32


# ✅ Parallel Reduction

In [5]:
import numpy as np
from numba import cuda
import time
    
@cuda.jit
def block_parallel_reduce(in_arr, out_arr):
    tid = cuda.threadIdx.x
    idx = cuda.grid(1) # global index
    n = len(in_arr)

    if idx >= n: # boundary check (matches CUDA C)
        return

    # pointer to this block's segment (logical equivalent to: in + blockIdx.x * blockDim.x)
    block_skip = cuda.blockIdx.x * cuda.blockDim.x

    # in-place reduction in global memory (interleaved / no divergence schema)
    # in-place reduction in global memory (stride doubles each step)
    stride = 1
    while stride < cuda.blockDim.x:
        if (tid % (2 * stride)) == 0:
            in_arr[block_skip + tid] += in_arr[block_skip + tid + stride]
        cuda.syncthreads()
        stride *= 2

    # write one value per block
    if tid == 0:
        out_arr[cuda.blockIdx.x] = in_arr[block_skip]

In [6]:
import numpy as np
from numba import cuda
import time
    
@cuda.jit
def block_parallel_reduce_no_divergence(in_arr, out_arr):
    tid = cuda.threadIdx.x
    idx = cuda.grid(1) # global index
    n = len(in_arr)

    if idx >= n: # boundary check (matches CUDA C)
        return

    # pointer to this block's segment (logical equivalent to: in + blockIdx.x * blockDim.x)
    array_offset = cuda.blockIdx.x * cuda.blockDim.x

    # in-place reduction in global memory (stride doubles each step)
    stride = 1
    while stride < cuda.blockDim.x:
        index = tid * 2 * stride
        if (index < cuda.blockDim.x):
            in_arr[array_offset + index] += in_arr[array_offset + index + stride]
        cuda.syncthreads()
        stride *= 2

    # write one value per block
    if tid == 0:
        out_arr[cuda.blockIdx.x] = in_arr[array_offset]

In [10]:
# ----------------------------
# host-side usage
# ----------------------------
blockSize = 1024;               # block dim 1D
numBlock = 1*1024*1024          # grid dim 1D
n = blockSize * numBlock;       # in_arr dim

print(f"Total elements: {n}")

# prepare data
a = np.ones(n, dtype=np.int32)
a_d = cuda.to_device(a)
b_d = cuda.device_array(numBlock, dtype=np.int32)

# numpy sum time
tic = time.time()
s_cpu = a.sum()
toc = time.time()
print(f"Numpy sum time: {toc - tic:.4f} seconds")

# launch kernel
t0 = time.perf_counter()
block_parallel_reduce[numBlock, blockSize](a_d, b_d)
cuda.synchronize()
t1 = time.perf_counter()
print(f"Kernel execution time: {t1 - t0:.4f} seconds")
print("speedup over numpy:", (toc - tic) / (t1 - t0))

# copy result back to host
b = b_d.copy_to_host()
s_gpu = b.sum()
print(f"Sum: {s_gpu} (Expected {s_cpu})")
assert s_cpu == s_gpu, "Error! Reduction result does not match!"

# print GPU memory info
used, free, total = mem_snapshot()
print("\nMemory occupancy:")
print(f"    GPU total: {total/1e9:.3f} GB")
print(f"    GPU free : {free/1e9:.3f} GB")
print(f"    GPU used : {used/1e9:.3f} GB")

Total elements: 1073741824
Numpy sum time: 0.6705 seconds
Kernel execution time: 0.1962 seconds
speedup over numpy: 3.4169335162911296
Sum: 1073741824 (Expected 1073741824)

Memory occupancy:
    GPU total: 15.828 GB
    GPU free : 11.417 GB
    GPU used : 4.411 GB


In [8]:
# ----------------------------
# host-side usage
# ----------------------------
blockSize = 1024;               # block dim 1D
numBlock = 1*1024*1024          # grid dim 1D
n = blockSize * numBlock;       # in_arr dim

print(f"Total elements: {n}")

# prepare data
a = np.ones(n, dtype=np.int32)
a_d = cuda.to_device(a)
b_d = cuda.device_array(numBlock, dtype=np.int32)

# numpy sum time
tic = time.time()
s_cpu = a.sum()
toc = time.time()
print(f"Numpy sum time: {toc - tic:.4f} seconds")

# launch kernel
t0 = time.perf_counter()
block_parallel_reduce_no_divergence[numBlock, blockSize](a_d, b_d)
cuda.synchronize()
t1 = time.perf_counter()
print(f"Kernel execution time: {t1 - t0:.4f} seconds")
print("speedup over numpy:", (toc - tic) / (t1 - t0))

# copy result back to host
b = b_d.copy_to_host()
s_gpu = b.sum()
print(f"Sum: {s_gpu} (Expected {s_cpu})")
assert s_cpu == s_gpu, "Error! Reduction result does not match!"

# print GPU memory info
used, free, total = mem_snapshot()
print("\nMemory occupancy:")
print(f"    GPU total: {total/1e9:.3f} GB")
print(f"    GPU free : {free/1e9:.3f} GB")
print(f"    GPU used : {used/1e9:.3f} GB")

Total elements: 1073741824
Numpy sum time: 0.6556 seconds
Kernel execution time: 0.0899 seconds
speedup over numpy: 7.289996969030649
Sum: 1073741824 (Expected 1073741824)

Memory occupancy:
    GPU total: 15.828 GB
    GPU free : 11.417 GB
    GPU used : 4.411 GB


## ↘️ TODO...

**Background: Divergence in Reduction**

-   Problem:
    -   threads in the same warp take different paths
    -   warps execute both paths (masked execution)
    -   performance drops
-   Goal:
    -   restructure indexing so the condition is based on a contiguous range of thread IDs

- Divergence-Avoiding Idea

    -  Instead of checking tid % (2\*stride) == 0, compute a new local index:
    $$
    index = 2 \cdot stride \cdot tid
    $$

    -   Then only threads with:
    $$
    index < blockDim.x
    $$


-   Implement the No-Divergence Reduction Loop

-   Use:

    -   `stride = 1, 2, 4, ...`
    -   `index = 2 * stride * tid`
    -   update: `in_arr[base + index] += in_arr[base + index + stride]`

Template:

```{python}
@cuda.jit
def blockParReduce_no_div(in_arr, out_arr, n):
    pass
```

# ✅ Image histogram

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
        
img = Image.open("../images/dog.png") # load image
img_mat = np.array(img).astype(np.uint8) # convert to numpy array
H, W, C = img_mat.shape
print(f"Image size: {W} x {H} x {C}")
img

## ↘️ TODO...

**Problem Description**

-   Given an RGB image `image` of shape: $(H, W, 3)$

-   compute a histogram such that:

    -   `histogram[0, i]` = num pixels with **red value** `i`
    -   `histogram[1, i]` = num pixels with **green value** `i`
    -   `histogram[2, i]` = num pixels with **blue value** `i`

-   Each color channel has **256 bins**

🔹 **CPU Reference Function**

-   CPU helper function that computes a frequency histogram for a 1D array:

``` python
def array_freq(arr):
    h = np.zeros(256, dtype=np.int32)
    for e in arr:
        h[e] += 1
    return h
```

🔹 **CUDA Kernel Requirements**

1.  Uses a 2D grid and 2D block
2.  Maps each thread to one pixel $(y, x)$
3.  Reads the pixel’s RGB values
4.  Updates the histogram using atomic additions
5.  Avoids out-of-bounds accesses

``` python
from numba import cuda

@cuda.jit
def histGPU(image, histogram):
    """
    image: uint8 array of shape (H, W, 3)
    histogram: int32 array of shape (3, 256)
    """
    # TODO
    
```